Universidad de Nariño<br>Maestría Ingeniería Electrónica<br>Reinforcement Learning<br><br><p style="font-size:30px">Taller 3 - Monte Carlo (MC): Pacman </p>   	|


**Objetivo:** encontrar la función de valor $q(s,a)$ y la política $\pi$ basado en métodos de control de Monte Carlo para una versión reducida del juego Pac-Man

# El entorno: Pac-Man
Pac-Man es un juego arcade muy famoso, lanzado por primera vez en 1980 y diseñado por Toru Iwatani. En esta tarea trabajaremos con un mapa reducido, en el cuál Pac-Man debe maximizar su score al comerce todas los «Power Pellets» mientras evade al fantasma Inky.<br>

<center>
    <img src="images/pacman_gui.PNG" alt="centered image" width=200/>
</center>

In [28]:
# Importar librerías
from pacman import create_pacman_env, reset_env, close_pacman
import random
import numpy as np
random.seed(0)

In [29]:
# Se crea el entorno de PACMAN
# Establecer la variable with_graphics en True/False según si se desea ejecutar el juego con gráficos o no
with_graphics = True 
create_pacman_env(render=with_graphics)

## Espacio de estados
El mapa consiste en un grid de 7x7, en donde en cada celda puede estar libre u ocupada con una pared, o un Power Pellet, o un agente (Pac-Man o fantasma).<br>

La siguiente celda contiene un list con todas las celdas sin pared, es decir, aquellas que pueden ser ocupadas tanto por el Pac-Man como por el fanstasma.

In [30]:
free_cells = [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 1), (2, 5), 
                (3, 1), (3, 3), (3, 5), (4, 1), (4, 3), (4, 5), (5, 1), 
                (5, 2), (5, 3), (5, 4), (5, 5)]
print(f'Cantidad de posiciones posibles para los agentes en el mapa: {len(free_cells)}')

Cantidad de posiciones posibles para los agentes en el mapa: 18


Para definir el estado del juego, basta con saber la ubicación del Pac-Man y del fantasma en el mapa, así como el estado de los dos power pellets (no comido/comido). Con esta definición tenemos:

* Cantidad de posibles posiciones del Pac-Man = 18
* Cantidad de posibles posiciones del fantasma = 18
* Estado de los dos power pellets = 4

Un estado del juego se describiría de la siguiente manera:
$s=\left((x_{pacman},y_{pacman}), (x_{ghost},y_{ghost}), (\textrm{No comido pp1}, \textrm{No comido pp2} )\right)$

La dimensión del espacio de estados sería:
$$|\mathcal{S}|=18\times 18 \times 4 = 1296$$

Para iniciar un nuevo juego utilizamos la función `reset_env(render=True/False )`

In [31]:
# Se (re)inicia el juego
env, state, _ = reset_env(render=with_graphics)
print(f'Estado al iniciar/resetear el juego, s={state}')

Estado al iniciar/resetear el juego, s=((2, 5), (3, 1), (True, True))


El estado inicial se muestra en la siguiente imagen:
<center>
    <img src="images/iniState2.png" alt="centered image" width=500/>
</center>

## Espacio de acciones
Las acciones disponibles para Pac-Man son moverse una celda hacia el norte, sur, este, oeste o no moverse. Sin embargo, estas 5 acciones pueden no ser válidas para todos los estados. Por ejemplo, para el estado incicial `s=((2, 5), (3, 1), (True, True))` no son válidas las acciones norte y sur, pues Pac-Man se chocaría con una pared al ejecutarlas.<br>

Vamos a obtener las acciones válidas para todas las posibles posiciones de Pac-Man, basados en la información de celdas libres en el mapa:

In [32]:
def get_legal_actions():
    actions = {}
    for cell in free_cells:
        actions[cell]=[]
        x = cell[0]+1
        y = cell[1]
        if (x,y) in free_cells:
            actions[cell].append('East')
        x = cell[0]-1
        y = cell[1]
        if (x,y) in free_cells:
            actions[cell].append('West')
        x = cell[0]
        y = cell[1]+1
        if (x,y) in free_cells:
            actions[cell].append('North')
        x = cell[0]
        y = cell[1]-1
        if (x,y) in free_cells:
            actions[cell].append('South')
        actions[cell].append('Stop')
    return actions


¿Cuáles serian las acciones válidas si Pac-Man está en la celda marcada con rojo?
<center>
    <img src="images/pacman_pp2.PNG" alt="centered image" width=150/>
</center>

In [33]:
# EDITABLE
# Ejemplo de cómo llamar a la función get_legal_actions() y obtener las acciones legales para la celda (3, 3)

# Ejecutamos la función para obtener el diccionario de acciones legales
legal_actions = get_legal_actions()

# Obtenemos las acciones disponibles para la celda (3, 3)
actions_for_cell = legal_actions.get((3, 3), "Celda no encontrada")

actions_for_cell
print(f'Acciones válidas si pac-man está en la celda (3, 3): {actions_for_cell}')

Acciones válidas si pac-man está en la celda (3, 3): ['East', 'Stop']


**Resultado esperado**
```
Acciones válidas si Pac-Man está en la celda (3, 3) = ['East', 'Stop']
```

# On-policy first-visit MC control

## Q-Values
Recuerde que los valores $Q^{*}(s,a)$ corresponden al valor esperado de la utilidad (recompensa) si el agente comienza en el estado $s$, toma la acción $a$ y de ahí en adelante se comporta siguiendo una política óptima. Estos valores se pueden aproximar por medio de Q-Value Iteration, así:<br>

$$Q_{k+1}^{*}(s,a)\leftarrow \sum_{s'} P(s'|s,a)(R(s,a,s')+\gamma \max_{a'}Q_{k}^{*}(s',a'))$$

Para esto se crea la tabla de Q-values, que guarda dichos valores para todos los estados y acciones posibles. Como cada estado puede tener diferente cantidad de acciones, se crea la tabla como un diccionario. Los Q-values se pueden inicializar de manera aleatoria. Pero para este caso, se van a inicializar en 0.0.

In [34]:
def create_q_table():    
    legal_actions = get_legal_actions()
    food_states = [(False, False),(False, True),(True, False),(True, True)]
    q_table = {}
    for cell_pacman in free_cells:
        q_table[cell_pacman] = {}
        for cell_ghost in free_cells:
            q_table[cell_pacman][cell_ghost] = {}
            for exist_food in food_states:
                q_table[cell_pacman][cell_ghost][exist_food] = {}
                for action in legal_actions[cell_pacman]:
                    q_table[cell_pacman][cell_ghost][exist_food][action] = 0.0
    return q_table

¿Cuáles serian los *Q_values* iniciales para el estado mostrado en la figura?


<center>
    <img src="images/pacman_qval.PNG" alt="centered image" width=150/>
</center>

In [35]:
#EDITABLE
q_table = create_q_table()
# Define el estado específico
estado_s = ((1, 4), (1, 3), (True, True))

# Extrae las posiciones de Pac-Man, del fantasma y el estado de la comida
cell_pacman, cell_ghost, food_state = estado_s

# Obtén los valores Q para todas las acciones posibles en el estado dado
q_values = q_table[cell_pacman][cell_ghost][food_state]

print(f'Q-values para el estado s={estado_s}: {q_values}')



Q-values para el estado s=((1, 4), (1, 3), (True, True)): {'North': 0.0, 'South': 0.0, 'Stop': 0.0}


**Resultado esperado**
```
Estado s=((1, 4),(1, 3),(True, True))
Q-values para el estado s = {'North': 0.0, 'South': 0.0, 'Stop': 0.0}
```

## Cómo escoger la acción?
Para que Pac-Man pueda aprender a ganar el juego debe poder explorar las consecuencias de aplicar diferentes acciones en los estados. Por esta razón es importante incorporar en la selección de la acción un método de exploración. En este caso trabajaremos con exporación $\epsilon-$greedy, la misma que se estudio en el **Taller 1: MAB**.

### Exploración $\epsilon-$greedy
Con probabilidad $(1-\epsilon)$ el agente **explota**. Es decir, escoge la acción con mayor utilidad esperada (acción *greedy*).
Con probabilidad $\epsilon$ el agente **explora**. Es decir, escoge una acción aleatoria entre las posibles acciones válidas para el estado actual.

Si el agente se encuentra en el estado $s$:
\begin{equation*}
a = \begin{cases}
\textrm{arg}\max_{a}Q(s,a) &\text{si $p>\epsilon$}\\
random &\text{si $p\leq \epsilon$}\\
\end{cases}
\end{equation*}

Implemente en la siguiente la función el algorimto para obtener la acción *greedy*

In [36]:
# EDITABLE
def get_policy_action(state):

    
    
    return best_action, value, possible_actions

In [37]:
# Define la función `get_policy_action` correctamente con dos parámetros
def get_policy_action(state, q_table, epsilon=0.1):
    """
    Devuelve una acción basada en la política epsilon-greedy para el estado dado.
    
    Parámetros:
        state: El estado actual como una tupla (posición de Pac-Man, posición de fantasma, estado de la comida).
        q_table: La tabla Q que contiene los valores Q para cada estado y acción.
        epsilon: La probabilidad de exploración (valor entre 0 y 1).
        
    Retorna:
        best_action: La acción seleccionada.
        value: El valor Q correspondiente a la acción seleccionada.
        possible_actions: Las acciones legales para el estado.
    """
    
    # Extraer las posiciones de Pac-Man, del fantasma y el estado de la comida en el estado dado
    cell_pacman, cell_ghost, food_state = state
    random.seed(0)
    # Obtener las acciones legales para el estado actual
    possible_actions = q_table[cell_pacman][cell_ghost][food_state]
    
    # Decidir si exploramos o explotamos
    if random.uniform(0, 1) > epsilon:  # Explotación (acción greedy)
        # Encuentra la acción con el valor Q más alto
        best_action = max(possible_actions, key=possible_actions.get)
        value = possible_actions[best_action]
    else:  # Exploración
        # Escoge una acción aleatoria de las acciones posibles
        best_action = random.choice(list(possible_actions.keys()))
        value = possible_actions[best_action]
    
    return best_action, value, possible_actions


Para probar la función, se pueden modificar artificialmente los valores Q para el estado `s=((2, 5), (3, 1), (True, True))`. 
* Q(s,'East') = 0.5
* Q(s,'West') = -0.15
* Q(s,'Stop') = 0.33

La acción greedy corresponde a aquella con mayor utilidad esperada, para este caso 'East'.
El valor del estado s correspondería al valor Q(s,'East')

In [38]:
# EDITABLE
q_table[state[0]][state[1]][state[2]] = {'East': 0.5, 'West': -0.15, 'Stop': 0.33}
# Estado específico para la prueba
test_state = ((2, 5), (3, 1), (True, True))
best_action, value, possible_actions = get_policy_action(test_state, q_table)
# Resultados esperados en el formato indicado
best_action, value, list(possible_actions.keys())
# Imprime los resultados esperados
print(f"Mejor acción según Q values: {best_action}")
print(f"Valor del estado s = {value}")
print(f"Posibles acciones para el estado s: {list(possible_actions.keys())}")

Mejor acción según Q values: East
Valor del estado s = 0.5
Posibles acciones para el estado s: ['East', 'West', 'Stop']


**Resultado esperado**
```
Mejor acción según Q values: East
Valor del estado s = 0.5
Posibles acciones para el estado s: ['East', 'West', 'Stop']
```

Ahora se va a implementar la función para la selección de la acción, incluyendo el método de exploración $\epsilon-$greedy.

Para seleccionar una acción aleatoria a partir de una lista de opciones, se puede usar `random.choice()`

In [39]:
# EDITABLE
def get_action(epsilon, state):

    
    
    return action

In [40]:
import random

# EDITABLE
def get_action(epsilon, state, q_table):
    """
    Selecciona una acción basada en la política epsilon-greedy.
    
    Parámetros:
        epsilon: Tasa de exploración.
        state: Estado actual como tupla (posición de Pac-Man, posición de fantasma, estado de la comida).
        q_table: Tabla Q que contiene los valores Q para cada estado y acción.
        
    Retorna:
        action: La acción seleccionada.
    """
    #random.seed(0)
    # Obtener las posibles acciones para el estado actual
    cell_pacman, cell_ghost, food_state = state
    possible_actions = q_table[cell_pacman][cell_ghost][food_state]
    
    # Decidir si explotamos o exploramos
    if random.uniform(0, 1) > epsilon:  # Explotación (acción greedy)
        action = max(possible_actions, key=possible_actions.get)
    else:  # Exploración
        action = random.choice(list(possible_actions.keys()))
    
    return action


Pruebe la función para el estado incial `s=((2, 5), (3, 1), (True, True))` con una tasa de exploración $\epsilon=0.2$.

In [41]:
# EDITABLE

# Estado específico para la prueba
test_state = ((2, 5), (3, 1), (True, True))

# Modificar los valores Q del estado test
q_table[test_state[0]][test_state[1]][test_state[2]] = {'East': 0.5, 'West': -0.15, 'Stop': 0.33}

# Tasa de exploración epsilon
epsilon = 0.2

# Probar la función para seleccionar la acción
selected_action = get_action(epsilon, test_state, q_table)

# Imprimir el resultado
print(f"Acción seleccionada para el estado {test_state}: {selected_action}")


Acción seleccionada para el estado ((2, 5), (3, 1), (True, True)): East


**Resultado esperado (con random.seed(0))**
```
Acción a tomar: East
```

## Generación de un episodio

Una vez listas las funciones para obtener la acción, incluyendo el mecanismo de exploración, se va a implementar una función que permita generar un episodio siguiendo la política $\pi: S_0, A_0, R_0, S_1, A_1, \cdots, S_{T-1}, A_{T-1}, R_{T}$

In [42]:
def gen_episode(env, state_0, epsilon):
    # TO DO: Implemente el ciclo para que se ejecute todo un episodio y guarde su información

    
    
    return history, rewards 

In [43]:
def gen_episode(env, state_0, epsilon):
    """
    Genera un episodio completo siguiendo la política epsilon-greedy.
    
    Parámetros:
        env: El entorno de Pac-Man.
        state_0: El estado inicial.
        epsilon: Tasa de exploración para la política epsilon-greedy.
    
    Retorna:
        history: Lista de tuplas (estado, acción) del episodio.
        rewards: Lista de recompensas obtenidas en el episodio.
    """
    # Inicializar listas para almacenar el historial del episodio y las recompensas
    history = []
    rewards = []
    
    # Estado inicial
    state = state_0
    done = False
    
    # Ejecutar el episodio
    while not done:
        # Seleccionar la acción según la política epsilon-greedy
        action = get_action(epsilon, state, q_table)
        
        # Ejecutar la acción en el entorno y obtener el siguiente estado, recompensa y si terminó el episodio
        next_state, reward, done, _ = env.step(action)
        
        # Guardar en el historial: (estado, acción) y almacenar la recompensa por separado
        history.append((state, action))
        
        # Almacenar la recompensa en la lista de recompensas
        rewards.append(reward)
        
        # Actualizar el estado actual
        state = next_state
    
    # Añadir un valor adicional al final de rewards para evitar el error de índice
    rewards.append(0)  # O cualquier otro valor que indique el final del episodio
    
    return history, rewards



In [44]:
# Se prueba la función con un epsilon=0.2
env, state, _ = reset_env(render=with_graphics)
#random.seed(0)
epsilon = 0.2
ep_history, ep_rewards = gen_episode(env, state, epsilon)

Pacman died! Score: -507


In [45]:
# Se imprimen los resultados
for i in range(len(ep_history)):
    print(f't={i}: (s,a)={ep_history[i]}, r={ep_rewards[i+1]}')

t=0: (s,a)=(((2, 5), (3, 1), (True, True)), 'East'), r=-1.0
t=1: (s,a)=(((3, 5), (4.0, 1.0), (True, True)), 'East'), r=-1.0
t=2: (s,a)=(((4, 5), (5.0, 1.0), (True, True)), 'East'), r=-1.0
t=3: (s,a)=(((5, 5), (5.0, 2.0), (True, True)), 'West'), r=-1.0
t=4: (s,a)=(((4, 5), (5.0, 3.0), (True, True)), 'East'), r=-1.0
t=5: (s,a)=(((5, 5), (5.0, 4.0), (True, True)), 'West'), r=-501.0
t=6: (s,a)=(((4, 5), (5.0, 5.0), (True, True)), 'East'), r=0


In [46]:
# Se imprimen los resultados
for i in range(len(ep_history)):
    print(f't={i}: (s, a)={ep_history[i]}, r={ep_rewards[i]}')


t=0: (s, a)=(((2, 5), (3, 1), (True, True)), 'East'), r=-1.0
t=1: (s, a)=(((3, 5), (4.0, 1.0), (True, True)), 'East'), r=-1.0
t=2: (s, a)=(((4, 5), (5.0, 1.0), (True, True)), 'East'), r=-1.0
t=3: (s, a)=(((5, 5), (5.0, 2.0), (True, True)), 'West'), r=-1.0
t=4: (s, a)=(((4, 5), (5.0, 3.0), (True, True)), 'East'), r=-1.0
t=5: (s, a)=(((5, 5), (5.0, 4.0), (True, True)), 'West'), r=-1.0
t=6: (s, a)=(((4, 5), (5.0, 5.0), (True, True)), 'East'), r=-501.0


**Resultado esperado (con random.seed(0))**
```
t=0: (s,a)=(((2, 5), (3, 1), (True, True)), 'East'), r=-1.0
t=1: (s,a)=(((3, 5), (4.0, 1.0), (True, True)), 'East'), r=-1.0
t=2: (s,a)=(((4, 5), (5.0, 1.0), (True, True)), 'East'), r=-1.0
t=3: (s,a)=(((5, 5), (5.0, 2.0), (True, True)), 'West'), r=-1.0
t=4: (s,a)=(((4, 5), (5.0, 3.0), (True, True)), 'East'), r=-1.0
t=5: (s,a)=(((5, 5), (4.0, 3.0), (True, True)), 'West'), r=-1.0
t=6: (s,a)=(((4, 5), (3.0, 3.0), (True, True)), 'East'), r=-1.0
t=7: (s,a)=(((5, 5), (4.0, 3.0), (True, True)), 'West'), r=-1.0
t=8: (s,a)=(((4, 5), (5.0, 3.0), (True, True)), 'East'), r=-1.0
t=9: (s,a)=(((5, 5), (5.0, 4.0), (True, True)), 'West'), r=-1.0
t=10: (s,a)=(((4, 5), (5.0, 5.0), (True, True)), 'East'), r=-501.0
```

## Implementación de MC de primera visita

### Algoritmo:
> Parámetro para exploración, $\epsilon>0$<br>
> Inicializar una política $\pi$ arbitraria, $Q(s,a)$ forma aleatoria<br>
> $Returns(s,a)$ $\leftarrow$ lista vacía<br>
> Para cada episodio:<br>
>> Generar un episodio siguiendo la política $\pi: S_0, A_0, R_0, S_1, A_1, \cdots, S_{T-1}, A_{T-1}, R_{T}$<br>
>> $G \leftarrow 0$ <br>
>> Para cada step del episodio, $t=T-1,T-2,\cdots,0$<br>
>>> $G \leftarrow \gamma G + R_{t+1}$<br>
>>> Si el par $(S_t, A_t)$ no aparece en $S_0, A_0, R_0, S_1, A_1, \cdots, S_{t-1}, A_{t-1}$: <br>
>>>> Agregar $G$ a la lista $Returns(s,a)$<br>
>>>> $Q(S_t,A_t) \leftarrow avg(Returns(s,a))$<br>
>>>> $A* \leftarrow argmax_a(Q(S_t,a))$<br>
>>>> Para todas las acciones $A \in \mathcal{A}(S_t)$

>>>>\[
\pi(a|S_t) \leftarrow 
\begin{cases} 
1 - \epsilon + \frac{\epsilon}{|\mathcal{A}(S_t)|} & \text{if } a = A^* \\ 
\frac{\epsilon}{|\mathcal{A}(S_t)|} & \text{if } a \neq A^* 
\end{cases} 
>>>>\]
>>>>> $\pi(a|S_t) \leftarrow \begin{cases} 1- \epsilon + \epsilon/|\mathcal{A}(S_t)| & \mbox{if } a=A* \\
\epsilon/|\mathcal{A}(S_t)| & \mbox{if } a \neq A* \end{cases} $

Realice un aprendizaje durante 1000 episodios, utilice una tasa de exploración de 0.05 y una tasa de descuento de 0.9. Para el entrenamiento, no ejecute el entorno con gráficas.

In [57]:
episodes = 1000
epsilon = 0.05 
gamma = 0.8 
with_graphics = False
Returns = {}

In [56]:
# Se crea la tabla q inicial
q_table = create_q_table()

# Ciclo que recorre los episodios generados
for ep in range(episodes) : 
    # Episodio actual
    print(f'Episode {ep}')
    
    # Se genera episodio
    env, state, _ = reset_env(render=with_graphics)
    ep_history, ep_rewards = gen_episode(env, state, epsilon)
    G = 0    
    
    # TO DO: Actualice la tabla q con el algoritmo de MC de primera visita

        

Episode 0
Pacman died! Score: -511
Episode 1
Pacman died! Score: -511
Episode 2
Pacman died! Score: -510
Episode 3
Pacman died! Score: -510
Episode 4
Pacman died! Score: -507
Episode 5
Pacman died! Score: -510
Episode 6
Pacman died! Score: -510
Episode 7
Pacman died! Score: -510
Episode 8
Pacman died! Score: -511
Episode 9
Pacman died! Score: -510
Episode 10
Pacman died! Score: -508
Episode 11
Pacman died! Score: -510
Episode 12
Pacman died! Score: -510
Episode 13
Pacman died! Score: -507
Episode 14
Pacman died! Score: -510
Episode 15
Pacman died! Score: -510
Episode 16
Pacman died! Score: -507
Episode 17
Pacman died! Score: -509
Episode 18
Pacman died! Score: -507
Episode 19
Pacman died! Score: -509
Episode 20
Pacman died! Score: -507
Episode 21
Pacman died! Score: -510
Episode 22
Pacman died! Score: -510
Episode 23
Pacman died! Score: -506
Episode 24
Pacman died! Score: -506
Episode 25
Pacman died! Score: -507
Episode 26
Pacman died! Score: -510
Episode 27
Pacman died! Score: -509
Ep

In [63]:
# Parámetros de entrenamiento
episodes = 1000
epsilon = 0.2
gamma = 0.5
with_graphics = False
Returns = {}

# Se crea la tabla q inicial
q_table = create_q_table()

# Ciclo que recorre los episodios generados
for ep in range(episodes):
    # Episodio actual
    print(f'Episode {ep}')
    
    # Se genera episodio
    env, state, _ = reset_env(render=with_graphics)
    ep_history, ep_rewards = gen_episode(env, state, epsilon)
    
    # Inicializar el retorno G y conjunto de visitas
    G = 0
    visited = set()
    
    # Recorre el episodio en orden inverso
    for t in reversed(range(len(ep_history))):
        # Obtener estado, acción y recompensa en el tiempo t
        state, action = ep_history[t]
        reward = ep_rewards[t]
        
        # Calcular el retorno acumulado G
        G = gamma * G + reward
        
        # Actualizar Q(s, a) solo si es la primera visita de (s, a)
        if (state, action) not in visited:
            visited.add((state, action))
            
            # Obtener el valor Q actual y el contador de visitas para el par (s, a)
            q_value = q_table[state[0]][state[1]][state[2]][action]
            
            # Actualizar el valor Q con promedio incremental
            if (state, action) not in Returns:
                Returns[(state, action)] = [G]
            else:
                Returns[(state, action)].append(G)
            
            # Promedio de retornos
            q_table[state[0]][state[1]][state[2]][action] = sum(Returns[(state, action)]) / len(Returns[(state, action)])


Episode 0
Pacman died! Score: -511
Episode 1
Pacman died! Score: -511
Episode 2
Pacman died! Score: -508
Episode 3
Pacman died! Score: -509
Episode 4
Pacman died! Score: -509
Episode 5
Pacman died! Score: -506
Episode 6
Pacman died! Score: -509
Episode 7
Pacman died! Score: -507
Episode 8
Pacman died! Score: -505
Episode 9
Pacman died! Score: -506
Episode 10
Pacman died! Score: -508
Episode 11
Pacman died! Score: -507
Episode 12
Pacman died! Score: -507
Episode 13
Pacman died! Score: -510
Episode 14
Pacman died! Score: -510
Episode 15
Pacman died! Score: -506
Episode 16
Pacman died! Score: -508
Episode 17
Pacman died! Score: -520
Episode 18
Pacman died! Score: -509
Episode 19
Pacman died! Score: -510
Episode 20
Pacman died! Score: -509
Episode 21
Pacman died! Score: -510
Episode 22
Pacman died! Score: -505
Episode 23
Pacman died! Score: -510
Episode 24
Pacman died! Score: -514
Episode 25
Pacman died! Score: -513
Episode 26
Pacman died! Score: -506
Episode 27
Pacman died! Score: -512
Ep

In [64]:
# Función para visualizar la tabla Q
def print_q_table(q_table):
    legal_actions = get_legal_actions()
    food_states = [(False, False),(False, True),(True, False),(True, True)]
    for cell_pacman in free_cells:
        for cell_ghost in free_cells:
            for exist_food in food_states:
                for action in legal_actions[cell_pacman]:
                    q_table[cell_pacman][cell_ghost][exist_food][action] = round(q_table[cell_pacman][cell_ghost][exist_food][action], 3)
                state = [cell_pacman, cell_ghost, exist_food]
                q_value = q_table[cell_pacman][cell_ghost][exist_food]
                print(f'Q(s, a): {state} {q_value}')

In [65]:
# Imprimir tabla Q final
print_q_table(q_table)

Q(s, a): [(1, 1), (1, 1), (False, False)] {'East': 0.0, 'North': 0.0, 'Stop': 0.0}
Q(s, a): [(1, 1), (1, 1), (False, True)] {'East': 0.0, 'North': 0.0, 'Stop': 0.0}
Q(s, a): [(1, 1), (1, 1), (True, False)] {'East': 0.0, 'North': 0.0, 'Stop': 0.0}
Q(s, a): [(1, 1), (1, 1), (True, True)] {'East': 0.0, 'North': 0.0, 'Stop': 0.0}
Q(s, a): [(1, 1), (1, 2), (False, False)] {'East': 0.0, 'North': 0.0, 'Stop': 0.0}
Q(s, a): [(1, 1), (1, 2), (False, True)] {'East': -53.452, 'North': -501.0, 'Stop': -501.0}
Q(s, a): [(1, 1), (1, 2), (True, False)] {'East': 0.0, 'North': 0.0, 'Stop': 0.0}
Q(s, a): [(1, 1), (1, 2), (True, True)] {'East': 0.0, 'North': 0.0, 'Stop': 0.0}
Q(s, a): [(1, 1), (1, 3), (False, False)] {'East': 0.0, 'North': 0.0, 'Stop': 0.0}
Q(s, a): [(1, 1), (1, 3), (False, True)] {'East': -20.553, 'North': -501.0, 'Stop': -125.752}
Q(s, a): [(1, 1), (1, 3), (True, False)] {'East': 0.0, 'North': 0.0, 'Stop': 0.0}
Q(s, a): [(1, 1), (1, 3), (True, True)] {'East': 0.0, 'North': 0.0, 'Stop':

Ahora ponga a prueba el agente de Pac-Man entrenado durante 5 episodios!

In [66]:
# Ciclo de 5 Episodios
for ep in range(5):
    # Se reinicia el entorno con la interfaz
    env, state, _ = reset_env(render=True)
    

    done = False
    # Ciclo de episodio
    while(not done):
        action, _, _ = get_policy_action(state,q_table)        
        new_state, reward, done, _ = env.step(action)                          
        if done:                       
            break            
        state = new_state

Pacman emerges victorious! Score: 487
Pacman emerges victorious! Score: 487
Pacman emerges victorious! Score: 487
Pacman emerges victorious! Score: 487
Pacman emerges victorious! Score: 487


In [53]:
close_pacman()

In [54]:
# Inicializar q_table como un diccionario vacío
q_table = {}

# Leer el archivo de texto línea por línea
with open('final_q_table.txt', 'r') as file:
    for line in file:
        # Verificar si la línea contiene el delimitador "){"
        if "Q(s, a):" in line and "){" in line:
            # Separar la parte de estado y la parte de acciones/valores Q
            state_part, action_part = line.split("){", 1)
            
            # Procesar la parte del estado
            state_str = state_part.split("Q(s, a): ")[1].strip() + ")"  # Agregar el paréntesis de cierre
            state = tuple(eval(state_str))  # Convertir a tupla para usarlo en el diccionario

            # Procesar la parte de acciones y valores Q
            action_str = "{" + action_part.strip()  # Agregar el paréntesis de apertura
            actions = eval(action_str)  # Convertir la cadena de acciones a un diccionario
            
            # Almacenar en q_table
            q_table[state] = actions
        else:
            print(f"Formato de línea inesperado o línea vacía: {line.strip()}")


Formato de línea inesperado o línea vacía: Q(s, a): [(1, 1), (1, 1), (False, False)]{'East': 0.0, 'North': 0.0, 'Stop': 0.0}
Formato de línea inesperado o línea vacía: Q(s, a): [(1, 1), (1, 1), (False, True)]{'East': 0.0, 'North': 0.0, 'Stop': 0.0}
Formato de línea inesperado o línea vacía: Q(s, a): [(1, 1), (1, 1), (True, False)]{'East': 0.0, 'North': 0.0, 'Stop': 0.0}
Formato de línea inesperado o línea vacía: Q(s, a): [(1, 1), (1, 1), (True, True)]{'East': 0.0, 'North': 0.0, 'Stop': 0.0}
Formato de línea inesperado o línea vacía: Q(s, a): [(1, 1), (1, 2), (False, False)]{'East': 0.0, 'North': 0.0, 'Stop': 0.0}
Formato de línea inesperado o línea vacía: Q(s, a): [(1, 1), (1, 2), (False, True)]{'East': 44.795, 'North': -501.0, 'Stop': -501.0}
Formato de línea inesperado o línea vacía: Q(s, a): [(1, 1), (1, 2), (True, False)]{'East': 0.0, 'North': 0.0, 'Stop': 0.0}
Formato de línea inesperado o línea vacía: Q(s, a): [(1, 1), (1, 2), (True, True)]{'East': 0.0, 'North': 0.0, 'Stop': 0.0}

Excelente! El Agente de Pac-Man entrenado gana el juego 5/5. Pero aun puede mejorar su política. Pruebe ajustando parámetros como tasa de exploración, de aprendizaje y de descuento. Incluso pruebe entrenando por mas tiempo. De igual forma, puede cambiar el random seed para obtener resultados diferentes.